In [97]:
import sys
import os
import re

cur_dir = "/Users/marshallkrassenstein/Desktop/projects/advent_of_code/"
os.chdir(cur_dir)

<article class="day-desc"><h2>--- Day 7: Handy Haversacks ---</h2><p>You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to <em>issues in luggage processing</em>.</p>
<p>Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!</p>
<p>For example, consider the following rules:</p>
<pre><code>light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
</code></pre>
<p>These rules specify the required contents for 9 bag types. In this example, every <code>faded blue</code> bag is empty, every <code>vibrant plum</code> bag contains 11 bags (5 <code>faded blue</code> and 6 <code>dotted black</code>), and so on.</p>
<p>You have a <code><em>shiny gold</em></code> bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one <code>shiny gold</code> bag?)</p>
<p>In the above rules, the following options would be available to you:</p>
<ul>
<li>A <code>bright white</code> bag, which can hold your <code>shiny gold</code> bag directly.</li>
<li>A <code>muted yellow</code> bag, which can hold your <code>shiny gold</code> bag directly, plus some other bags.</li>
<li>A <code>dark orange</code> bag, which can hold <code>bright white</code> and <code>muted yellow</code> bags, either of which could then hold your <code>shiny gold</code> bag.</li>
<li>A <code>light red</code> bag, which can hold <code>bright white</code> and <code>muted yellow</code> bags, either of which could then hold your <code>shiny gold</code> bag.</li>
</ul>
<p>So, in this example, the number of bag colors that can eventually contain at least one <code>shiny gold</code> bag is <code><em>4</em></code>.</p>
<p><em>How many bag colors can eventually contain at least one <code>shiny gold</code> bag?</em> (The list of rules is quite long; make sure you get all of it.)</p>
</article>?

In [98]:
import pandas as pd

# list[pd.read_table('input_numbers.txt')]
# f = open("input_numbers.txt", "r")
def read_file(file_path):
    f= open(file_path, 'r')
    final_list=[]
    for line in f:
        final_list.append(line.replace('\n', ''))
    return final_list
bag_rules = read_file('day_7/bag_rules.txt')
print(bag_rules[:5])

['shiny plum bags contain no other bags.', 'clear crimson bags contain 3 pale aqua bags, 4 plaid magenta bags, 3 dotted beige bags, 3 dotted black bags.', 'dim violet bags contain 5 bright brown bags.', 'mirrored tomato bags contain 3 faded maroon bags, 3 dark green bags.', 'muted salmon bags contain 1 posh yellow bag.']


In [3]:

def extract_rules(rule_list):
    from collections import defaultdict

    rule_dict = defaultdict(list)
    
    for item in rule_list:
        p = re.compile('[a-z]+\s[a-z]+\sbag')
        extract = p.findall(item)

        if len(extract) <= 1:
            pass
        else:
            rule_dict[extract[0]] += extract[1:] 
            rule_dict[extract[0]] = set(rule_dict[extract[0]])
    return rule_dict

rules_dict = extract_rules(bag_rules)

In [4]:
def recurse_rules(rule_dict):

    checked_bags = []
    gold_bags = []
    
    def recurse_func(key_bag, bags, layer): 
        if 'no other bag' in bags:
            return
        if 'shiny gold bag' in bags:
            gold_bags.append(key_bag)
        for new_rule in bags:
            if new_rule in gold_bags:
                gold_bags.append(key_bag)
            if new_rule in checked_bags:
                pass
            else:
                if new_rule in rule_dict:
                    recurse_func(new_rule, rule_dict[new_rule], layer+1)
                    checked_bags.append(new_rule)
        return
    count_iter = 0
    while True:
        
        tot_len = len(set(gold_bags))
        count_iter +=1
        for key_bag in rule_dict:
            recurse_func(key_bag, rule_dict[key_bag], 1)
        if tot_len == len(set(gold_bags)):
            print(f"Success found in {count_iter} recursive iterations")
            return count_iter, len(set(gold_bags))
        print(tot_len, len(set(gold_bags)))
    return len(set(gold_bags))

recurse_rules(rules_dict)

0 68
68 104
104 118
118 127
127 128
Success found in 6 recursive iterations


(6, 128)

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!</p>
<p>Consider again your <code>shiny gold</code> bag and the rules from the above example:</p>
<ul>
<li><code>faded blue</code> bags contain <code>0</code> other bags.</li>
<li><code>dotted black</code> bags contain <code>0</code> other bags.</li>
<li><code>vibrant plum</code> bags contain <code>11</code> other bags: 5 <code>faded blue</code> bags and 6 <code>dotted black</code> bags.</li>
<li><code>dark olive</code> bags contain <code>7</code> other bags: 3 <code>faded blue</code> bags and 4 <code>dotted black</code> bags.</li>
</ul>
<p>So, a single <code>shiny gold</code> bag must contain 1 <code>dark olive</code> bag (and the 7 bags within it) plus 2 <code>vibrant plum</code> bags (and the 11 bags within <em>each</em> of those): <code>1 + 1*7 + 2 + 2*11</code> = <code><em>32</em></code> bags!</p>
<p>Of course, the actual rules have a <span title="100%">small</span> chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!</p>
<p>Here's another example:</p>
<pre><code>shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
</code></pre>
<p>In this example, a single <code>shiny gold</code> bag must contain <code><em>126</em></code> other bags.</p>
<p><em>How many individual bags are required inside your single <code>shiny gold</code> bag?</em></p>
</article>

In [75]:
def extract_more_rules(rule_list):
    from collections import defaultdict

    rule_dict = {}
    for item in rule_list:
#         print(item)
        p1 = re.compile('^[a-z]+\s[a-z]+')
        p_rest = re.compile('\d\s[a-z]+\s[a-z]+')
        extract_key = p1.findall(item)[0]
        extract_rule = p_rest.findall(item)
#         print(extract_key, extract_rule)
        if len(extract_key) == 0 or len(extract_rule)==0:
#             print('not a rule')
            pass
        else:
            p2 = re.compile('^\d+\s')
            p3 = re.compile('[a-z]+\s[a-z]+$')
            in_dict = {}
            for i in extract_rule:
                extract_num = int(p2.findall(i)[0])
                extract_in_key = p3.findall(i)[0]
                in_dict[extract_in_key] = extract_num
#                 print(extract_in_key, extract_num)
            rule_dict[extract_key] = in_dict
            
    return rule_dict

num_rules_dict = extract_more_rules(bag_rules)

In [95]:
def find_total_bag(rule_dict, bag_color):
    
    #colored bags
    colored_bags = []
    
    def nest_count(rule_dict, bag_color, count):
        if bag_color not in rule_dict:
            print(f'Made it to end with {count} bags')
            return {bag_color: count}
        start_nest = rule_dict[bag_color]
#         local_count = sum(start.values()) * layer
        print(start_nest)
        for item in start_nest:
#             local_count = start_nest[item]*count
            print(f'recursing into bag {item} with count {start_nest[item]*count}')
            print(start_nest[item], count)
            colored_bags.append({item: start_nest[item]*count})
            nest_count(rule_dict, item, start_nest[item]*count)
    
    start = rule_dict[bag_color]
    for item in start:
        colored_bags.append({item: start[item]})
        nest_count(rule_dict, item, start[item])

    return sum([sum(bag.values()) for bag in colored_bags])
    

find_total_bag(num_rules_dict, 'shiny gold')
# print(num_rules_dict)

{'posh magenta': 5, 'light aqua': 2, 'wavy purple': 3}
recursing into bag posh magenta with count 15
5 3
{'plaid plum': 4, 'vibrant lime': 2, 'light aqua': 5, 'dull blue': 2}
recursing into bag plaid plum with count 60
4 15
Made it to end with 60 bags
recursing into bag vibrant lime with count 30
2 15
Made it to end with 30 bags
recursing into bag light aqua with count 75
5 15
{'vibrant lime': 1, 'clear gold': 3, 'plaid plum': 1, 'shiny plum': 5}
recursing into bag vibrant lime with count 75
1 75
Made it to end with 75 bags
recursing into bag clear gold with count 225
3 75
Made it to end with 225 bags
recursing into bag plaid plum with count 75
1 75
Made it to end with 75 bags
recursing into bag shiny plum with count 375
5 75
Made it to end with 375 bags
recursing into bag dull blue with count 30
2 15
Made it to end with 30 bags
recursing into bag light aqua with count 6
2 3
{'vibrant lime': 1, 'clear gold': 3, 'plaid plum': 1, 'shiny plum': 5}
recursing into bag vibrant lime with coun

20189

In [96]:
test = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags."""

rule = test.split('\n')
rule_test = extract_more_rules(rule)
find_total_bag(rule_test, 'shiny gold')

{'dark orange': 2}
recursing into bag dark orange with count 4
2 2
{'dark yellow': 2}
recursing into bag dark yellow with count 8
2 4
{'dark green': 2}
recursing into bag dark green with count 16
2 8
{'dark blue': 2}
recursing into bag dark blue with count 32
2 16
{'dark violet': 2}
recursing into bag dark violet with count 64
2 32
Made it to end with 64 bags


126